In [26]:
import sys
import time
import pandas as pd
sys.path.append('/home/saulo/projects/cpl/src')

from cpl.utils import db_onto, db_ap, flatten
from cpl.sscrowd import sscrowd_scores
from cpl.promotion import promote_seeds, all_promoted_patterns, all_promoted_instances

In [27]:
iteration = 1
max_promotions = 15
limit = 5000
T = 3

category = db_onto.find_one({'category_name':'fruit'})
        
df_all_promoted_instances = all_promoted_instances()
df_all_promoted_patterns = all_promoted_patterns()

In [35]:
sscrowd_s = sscrowd_scores()

tot_s = time.time()

last_promoted_patterns = category['promoted_patterns'][iteration-1]
promoted_instances = list(flatten(category['promoted_instances']))
promoted_patterns = list(flatten(category['promoted_patterns']))

#get the promoted patterns of the mutex exception categories
mutex_query = (db_onto.ontology.
               find({'category_name':{"$in":category['mutex_exceptions']}}))

mutex_patterns = list(flatten([i['promoted_patterns'] for i in mutex_query]))

#extraction step
#count the ocurrences of instances that co-occur with the
#positive promoted patterns in the last iteration
#without considering instances that were already promoted
pos = list(db_ap.find({"ctx_pattern":{"$in":last_promoted_patterns},
                                       "noun_phrase":{"$nin":promoted_instances},
                                       "counter":{"$gt":1}}))

if (pos):  #if at least one positive and one negative pattern was found

    df_pos = (pd.DataFrame(pos)
              .join(sscrowd_s[sscrowd_s.category_name == category['category_name']]['score'], on='ctx_pattern')
              .fillna(1)
              .assign(counter=lambda df: (df['counter']*df['score']))
              .groupby('noun_phrase')
              ['counter']
              .sum()
              .sort_values(ascending=False)
              .head(limit)
              .rename('count_pos'))

    print(df_pos)

    #count the ocurrences of instances that co-occur with negative patterns
    df_neg = (df_all_promoted_patterns
              [~df_all_promoted_patterns['ctx_pattern']
               .isin(promoted_patterns + mutex_patterns)]
              .groupby('noun_phrase')
              ['counter']
              .sum()
              .rename('count_neg'))

    joined = (pd.concat([df_pos,df_neg], axis=1, sort=False)
              .fillna(0)
              .assign(filter_check=lambda df:  
                      (df['count_pos'] >= df['count_neg']*T) &  # filter criterion #1
                      (df['count_pos'] >= 2)))                  # filter criterion #2

    new_instances=list((joined[joined['filter_check']]                 # filter step
                        .sort_values(by='count_pos', ascending=False)  # rank step
                        .head(max_promotions)                          # promote step
                        .index.values))

    #update ontology with the promoted instances
#    db_cpl.ontology.update_one({'category_name':category['category_name']},
#                               {'$set':{'promoted_instances.'+str(iteration):new_instances}})
else:
    new_instances = []
    
tot_e = time.time()
print(tot_e - tot_s)

print(new_instances)

noun_phrase
bananas            46.0
apple              16.0
tomatoes           14.0
almonds             8.0
guava               6.0
cape gooseberry     6.0
peach               6.0
papayas             6.0
melons              6.0
lychee              6.0
raisins             6.0
berries             6.0
cranberries         6.0
coconuts            6.0
rambutan            4.0
apricot             4.0
avocados            4.0
cabbage             4.0
cactus              4.0
pomegranates        4.0
plum                4.0
grapefruits         4.0
sour cherries       4.0
lanzones            4.0
pear                4.0
persimmons          4.0
plantains           4.0
durians             4.0
Name: count_pos, dtype: float64
0.033641815185546875
['bananas', 'almonds', 'guava', 'cape gooseberry', 'peach', 'papayas', 'melons', 'lychee', 'raisins', 'cranberries', 'grapefruits', 'plantains', 'persimmons', 'pear', 'lanzones']


In [32]:
df_pos

noun_phrase
bananas            23.0
apple               8.0
tomatoes            7.0
almonds             4.0
guava               3.0
cape gooseberry     3.0
peach               3.0
papayas             3.0
melons              3.0
lychee              3.0
raisins             3.0
berries             3.0
cranberries         3.0
coconuts            3.0
rambutan            2.0
apricot             2.0
avocados            2.0
cabbage             2.0
cactus              2.0
pomegranates        2.0
plum                2.0
grapefruits         2.0
sour cherries       2.0
lanzones            2.0
pear                2.0
persimmons          2.0
plantains           2.0
durians             2.0
Name: count_pos, dtype: float64

In [121]:
s1 = pd.Series([40,50,60,70,80],index=['pira','americana','sao paulo','sao carlos','marilia']).rename('maluco')
s2 = pd.Series([5,35,25,15], index=['pira','sao paulo','sao carlos','americana'])
s3 = pd.Series([])

x = pd.concat([s1,s3], axis=1, sort=False)

x

,maluco,0
pira,40,NaN
americana,50,NaN
sao paulo,60,NaN
sao carlos,70,NaN
marilia,80,NaN


In [20]:
last_promoted_instances = category['promoted_instances'][iteration-1]
all_promoted_patterns = list(flatten(category['promoted_patterns']))
promoted_instances = list(flatten(category['promoted_instances']))

#get the promoted instances of the mutex exception categories
mutex_query = (db_onto.
               find({'category_name':{"$in":category['mutex_exceptions']}}))

mutex_instances = list(flatten([i['promoted_instances'] for i in mutex_query]))

#extraction step
#count the ocurrences of patterns that co-occur with the
#positive promoted instances in the last iteration
#without considering patterns that were already promoted
pos = list(db_ap.find({"noun_phrase":{"$in":last_promoted_instances},
                                       "ctx_pattern":{"$nin":all_promoted_patterns},
                                       "counter":{"$gt":1}}))

if (pos):  #if at least one positive and one negative instance was found
    df_pos = (pd.DataFrame(pos)
              .groupby('ctx_pattern')
              ['counter']
              .sum()
              .sort_values(ascending=False)
              .head(limit)
              .rename('count_pos'))
    
    #count the ocurrences of patterns that co-occur with negative instances
    df_neg = (df_all_promoted_instances
              [~df_all_promoted_instances['noun_phrase'].isin(promoted_instances + mutex_instances)]
              .groupby('ctx_pattern')
              ['counter']
              .sum()
              .rename('count_neg'))

    joined = (pd.concat([df_pos, df_neg], axis=1, sort=False)
              .fillna(0)
              .assign(filter_check=lambda df:  
                      (df['count_pos'] >= df['count_neg']*T) &   # filter criterion #1
                      (df['count_pos'] >= 2)))                   # filter criterion #2
    
    df_count = (df_all_promoted_instances
                [df_all_promoted_instances['noun_phrase'].isin(promoted_instances)]
                .groupby('ctx_pattern')
                ['counter']
                .sum()
                .rename('count_cnt'))
    
    joined = (joined.join(df_count)
              .fillna(0)
              .assign(precision=lambda df:
                      df['count_pos']/df['count_cnt']))  #rank criterion

    promoted_patterns = list((joined[joined['filter_check']]                 # filter step                                  
                              .sort_values(by='precision', ascending=False)  # rank step
                              .head(max_promotions)                          # promote step
                              .index.values))

    #update ontology with the promoted patterns
#    db_onto.update_one({'category_name':category['category_name']},
#                               {'$set':{'promoted_patterns.'+str(iteration):promoted_patterns}})

else:
    promoted_patterns = []
    
print(promoted_patterns)

[]


In [22]:
joined.sort_values(by=['precision','count_pos'], ascending=[False,False])

,count_pos,count_neg,score,filter_check,count_cnt,precision
made from _,0.0,1597.0,0.0,False,259.0,0.0
wine made from _,0.0,0.0,0.0,False,56.0,0.0
produced from _,0.0,60.0,0.0,False,63.0,0.0
_ destined for,0.0,117.0,0.0,False,47.0,0.0
fruit such as _,0.0,7.0,0.0,False,102.0,0.0
crops such as _,0.0,287.0,0.0,False,76.0,0.0
_ grown on,0.0,22.0,0.0,False,35.0,0.0
_ are grown in,0.0,22.0,0.0,False,45.0,0.0
made with _,0.0,519.0,0.0,False,68.0,0.0
made of _,0.0,5639.0,0.0,False,47.0,0.0


In [98]:
df_x_neg[df_x_neg]

,_id,counter,ctx_pattern,noun_phrase
0,5cb09998014f1f866562b1c1,18.0,Born in _,a coruña
1,5cb09998014f1f866562b1c6,2.0,Galician city of _,a coruña
2,5cb09998014f1f866562b1d0,2.0,Spanish city of _,a coruña
3,5cb09998014f1f866562b1d2,10.0,Spanish province of _,a coruña
4,5cb09998014f1f866562b1d3,2.0,Spanish provinces of _,a coruña
5,5cb09998014f1f866562b1d5,4.0,Vuelta began in _,a coruña
6,5cb09998014f1f866562b1f8,2.0,_ known as,a coruña
7,5cb09998014f1f866562b1ff,2.0,_ playing another year,a coruña
8,5cb09998014f1f866562b20a,2.0,_ was founded in,a coruña
9,5cb09998014f1f866562b218,4.0,arrived in _,a coruña


In [11]:
def sscrowd():
    columns = ['category','ctx_pattern','score']
    data = [['city','cities including _','0.8']]
    
    df_sscrowd = pd.DataFrame(data=data, columns=columns).set_index('ctx_pattern')
    
    return df_sscrowd

x = sscrowd()

In [12]:
x

,category,ctx_pattern,score
0,city,cities including _,0.8


In [13]:
x = sscrowd_scores()
x[x.category_name == 'fruit']

,_id,category_name,score
ctx_pattern,,,
_ grown in,5cb49e4e10d4cc4a867a6a1e,fruit,1
most wineries today the _,5cb49e5910d4cc4a867a6a1f,fruit,0
translated as full of _,5cb49e5e10d4cc4a867a6a20,fruit,0
